# Обработка текстов

## Регулярные выражения

Часто возникает задача поиска в тексте каких-то элементов (например гиперссылок) или проверки введенных пользоватем данных. Для упрощения этих задач можно использовать регулярные выражения, реализация которых присутствует в стандартных библиотеках большей части популярных языков программирования, в том числе и в `Python`.

Наиболее типичные сценария использования:

* поиск паттерна в строке
* проверка строки на совпадение паттерну 
* сегментация строки по паттерну 
* замена паттерна в строке.

Паттерн описывается с помощью специального языка - регулярного выражения, в `Python` используются регулярные выражения со следующими базовыми наборами правил:

|           |                       |              |                               |
|-----------|-----------------------|--------------|-------------------------------|
| .         | любой символ          | \\d          | цифра                         |
| \\D       | не цифра              | \\s          | пробельный символ             |
| \\S       | не пробельный символ  | \\w          | буквенный символ              |
| \\W       | не буквенный символ   | ^            | начало строки                 |
| $         | конец строки          | \\b          | начало слова                  |
| \\B       | конец слова           | \[abc\]      | символ из перечисленных       |
| \[^abc\]  | кроме символов        | \[a\-zA\-Z\] | символы из интервалов         |
| X\|Y      | или                   |


Наборы символов и символьных классов можно объединять в группы, наподобие того как это происходит в обычных математических выражениях.  Для описания групп с различными целями существует следующие конструкции

|           |                      |           |                           |
|-----------|----------------------|-----------|---------------------------|
| \(X\)     | группа \(capturing\) | \(?:X\)   | группа \(non\-capturing\) |
| \(?=X\)   | предпросмотр         | \(?\!X\)  | негативный предпросмотр   |

Квантификатор после какого-то подвыражения (символа или группы) позволяет задать правила повторения этого подвыражения. Квантификатор может относиться более чем к одному символу в регулярном выражении только если это группа.  В `Python` определены жадные квантификаторы 

|          |                       |           |                          |
|----------|-----------------------|-----------|--------------------------|
| X?       | 0 или 1 повторение    | X\*       | $\geq0$ повторений       |
| X\+      | $\geq 1$ повторений   | X\{n\}    | ровно $n$ повторений     |
| X\{n,\}  | $\geq n$ повторений   | X\{n,m\}  | от $n$ до $m$ повторений |


и ленивые квантификаторы

|          |                       |           |                          |
|----------|-----------------------|-----------|--------------------------|
| X??      | 0 или 1 повторение    | X\*?      | $\geq0$ повторений       |
| X\+?     | $\geq 1$ повторений   | X\{n\}?   | ровно $n$ повторений     |
| X\{n,\}? | $\geq n$ повторений   | X\{n,m\}? | от $n$ до $m$ повторений |

Разница заключается в алгоритме сопоставления - жадные квантификаторы пытаются сопоставить как можно больше символов входной строки, ленивые - как можно меньше.



## Реализации регулярных выражений

### Формальные языки

**Формальный язык** — множество конечных слов над конечным алфавитом $\Sigma$. 
Пусть есть некоторое конечно множество символов $\Sigma$, тогда множество $L \in \Sigma^*$ есть формальный язык. 

Над формальными языками можно определить операции:

* $L_1 \cap L_2$
* $L_1 \cup L_2$
* $L_1 \setminus L_2$
* $L_1 \cdot L_2 $ - новый язык, в котором ко всем возможным словам из $L_1$ присоеденены справа слова из $L_2$
* $L^*$ - замыкание клини, $\{\epsilon\} \cup L \cup (L \cdot L) \cup (L \cdot L \cdot L) \cup \cdots$

Формальный язык над алфавитом $\Sigma$ является **регулярным**, если он принадлежит множеству языков $R \in \Sigma^*$:

* $\varnothing \in R$
* $\{\varepsilon\} \in R$
* $\forall a \in \Sigma: \{a\} \in R$
* $P \in R \land Q \in R \Rightarrow (P \cup Q) \in R$
* $P \in R \land Q \in R \Rightarrow (P \cdot Q) \in R$
* $P \in R \Rightarrow P^* \in R$

Любой регулярный язык может быть описан:

* детерменированным конечным автоматом
* недетерменированным конечным автоматом
* регулярным выражением
* регулярном грамматикой

**Конечные автоматы**

Конечный автомат это упорядоченная пятерка $A = (\Sigma, Q, q_0, F, \delta)$, где

* $\Sigma$ - входной алфавит
* $Q$ - множество состояний
* $q_0 \in Q$ - начальное состояние
* $F \subset Q$ - множество конечных состояний
* $\delta: (\Sigma \cup \varepsilon) \times Q \rightarrow 2^Q$ - функция перехода

В зависимости от определения функции перехода:

* недетерминированный конечный автомат с $\varepsilon$-переходами ($\varepsilon$-NFA)
$$\delta: (\Sigma \cup \varepsilon) \times Q \rightarrow 2^Q$$ 
* недетерминированный конечный автомат (NFA)
$$\delta: \Sigma \times Q \rightarrow 2^Q$$ 
* детерминированный конечный автомат (DFA)
$$\delta: \Sigma \times Q \rightarrow Q$$ 

Множество слов, которые принимаются конечным автоматом образуют регулярный язык. По любому $\varepsilon$-NFA можно построить эквивалентный DFA. В DFA можно минимизировать число состояний. 

Графическое отображение конечного автомата представлено на изображении

![конечный автомат](img/dfa.svg)


Существует два распространенных способа реализации регулярных выражений, применяемых в различных задачах (не считая гибридов и т.п.):

1. регулярное выражение -> $\varepsilon$-NFA -> DFA -> min-DFA
2. backtracking 

В стандартных библиотеках популярных языков программирования применяется backracking.

## Разновидности синтаксиса регулярных выражений

Исторически существует несколько возможных  синтаксисов языков для описания регулярных выражений

* POSIX RE[@posix-re-ref] (`., *, [ ], [^ ], \{ \}, \( \)`)
* POSIX ERE (`., *, +, ?, |, [ ], [^ ], { }, ( )`)
* PCRE[@pcre-ref] (стандартные библиотеки `Perl`, `Java`, `C#`, `Python`)

## Реализация в языке Python

В `Python` стандартные регулярные выражения определены в модуле `re`. Кроме того, существует специальный синтаксис задания строк для регулярных выражений - префикс `r`. В строках с этим префиксом не требуется специальным образом задавать специальные управляющие символы (например, `,`)


In [1]:
import re

Рекомендует сначала скомпилировать регулярное выражение с помощью функции `re.compile()`, в результате получится специальный объект, который можно использовать для выполнения основных задач

In [2]:
pattern = re.compile(r'a+a$')

Например, чтобы проверить совпадает ли паттерн, заданный регулярным выражением, со строкой, можно использовать метод `fullmatch`


In [3]:
print(pattern.fullmatch('aaaa'))

<re.Match object; span=(0, 4), match='aaaa'>


В данном случае строка `'aaaa'` соответствует паттерну `'a+a$'`, возвращается объект специального типа. Если соответствия нет, то возвращается `None`.

Второй возможный сценарий использования - поиск непересекающихся подстрок, которые соответствую паттерну. Введем паттерн - все слова из букв *a* которые заканчиваются на *b*.

In [4]:
pattern = re.compile(r'a+b')

Найдем все вхождения этого паттерна в строке:

In [5]:
s = 'Hello aaab world ab !'
pattern.findall(s)

['aaab', 'ab']

Ещё один сценарий - разделение строки на подстроки по паттерну. Это бывает полезным, когда нужно быстро разделить, например, предложения на слова, убрав все знаки препинания. 


In [6]:
pattern.split(s)

['Hello ', ' world ', ' !']

Следующий сценарий использования - замена вхождений паттерна в строке.


In [7]:
pattern.sub('e', s)

'Hello e world e !'


В качестве замены можно передать не только строку, но и `lambda`-функцию, которая будет вызываться на каждое совпадение с образцом. В данном случае мы добавляем к каждому совпадению букву *a*.

In [8]:
pattern.sub(lambda x: x.group(0) + 'a', s)

'Hello aaaba world aba !'


Приведем более сложный пример. В качестве паттерна опишем адрес электронной почты (упрощенный, данный паттерн не удовлетворяет спецификации).Дальше разобьем этот адрес на составляющие:

In [9]:
pattern = re.compile(r'(\w+)@(\w+)\.(\w{2,3})')

matcher = pattern.match('test@example.com')
if matcher:
    print(matcher.group(0))
    print(matcher.group(1))
    print(matcher.group(2))
    print(matcher.group(3))

test@example.com
test
example
com


Можно найти все вхождения электронной почты в тексте и напечатать только домены первого уровня:


In [10]:
t = 'test@example.com ssss test2@gmail.com'

pattern = re.compile(r'(\w+)@((\w+)\.(\w{2,3}))')
for m in pattern.finditer(t):
    print (m.group(2))

example.com
gmail.com


или имена пользователей:

In [11]:
s = 'test@example.com hello@mail.ru'
matchers = pattern.finditer(s)
for matcher in matchers:
    print (matcher.group(1))

test
hello


Обратим внимание на работу жадных и ленивых квантификаторов. В случае применения жадных результат может быть не тем, что ожидается


In [12]:
s = '<h1> text1 </h1>  <h2> text3 </h2>'
re.findall(r'<\w+>(?:.+)</\w+>', s)

['<h1> text1 </h1>  <h2> text3 </h2>']

В данном случае можно использовать ленивые

In [13]:
re.findall(r'<\w+>(?:.+?)</\w+>', s)

['<h1> text1 </h1>', '<h2> text3 </h2>']

Ещё один важный момент касается производительности движка регулярных выражений. Так как в `Python` регулярные выражения реализованы через механизм backtracking'а, то в некоторых случаях можно получить экспоненциальную сложность. 

In [14]:
re.fullmatch(r'(a*a*)*c', 'a' * 10 + 'e')